In [1]:
# Import Packages
from sqlalchemy import create_engine
import pandas as pd

In [132]:
# Create engine with database
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

* Q10:
Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [131]:
with engine.connect() as con:
    rs = con.execute("SELECT f.name AS Facility, \
                        SUM(CASE WHEN b.memid = 0 THEN (b.slots * f.guestcost) \
                                 ELSE (slots * membercost) END) AS Revenue \
                        FROM Bookings AS b \
                        INNER JOIN Facilities AS f \
                        ON b.facid = f.facid \
                        GROUP BY Facility \
                        ORDER BY Revenue DESC")
    
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

In [23]:
df

,Facility,Revenue
0,Massage Room 1,50351.6
1,Massage Room 2,14454.6
2,Tennis Court 2,14310.0
3,Tennis Court 1,13860.0
4,Squash Court,13468.0
5,Badminton Court,1906.5
6,Pool Table,270.0
7,Snooker Table,240.0
8,Table Tennis,180.0


* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [69]:
with engine.connect() as con:
    rs = con.execute("SELECT m2.firstname || ' ' ||m2.surname AS Name, m1.surname AS RecommendedByLastName, m1.firstname AS RecommendedByFirstName \
                     FROM Members AS m1 \
                     INNER JOIN Members AS m2 \
                     ON m1.memid = m2.recommendedby \
                     ORDER BY RecommendedByLastName, RecommendedByFirstName")
    
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

In [71]:
df.head()

,Name,RecommendedByLastName,RecommendedByFirstName
0,Ramnaresh Sarwin,Bader,Florence
1,Joan Coplin,Baker,Timothy
2,Matthew Genting,Butters,Gerald
3,Timothy Baker,Farrell,Jemima
4,David Pinker,Farrell,Jemima


* Q12: Find the facilities with their usage by member, but not guests

In [173]:
with engine.connect() as con:
    rs = con.execute("SELECT f.name AS Facility, SUM(b.slots * 30) / 60 AS Hours_Booked \
                     FROM Bookings AS b \
                     LEFT JOIN Facilities AS f \
                     ON b.facid = f.facid \
                     WHERE memid != 0 \
                     GROUP BY Facility")
    
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

In [174]:
df

,Facility,Hours_Booked
0,Badminton Court,543
1,Massage Room 1,442
2,Massage Room 2,27
3,Pool Table,428
4,Snooker Table,430
5,Squash Court,209
6,Table Tennis,397
7,Tennis Court 1,478
8,Tennis Court 2,441


* Q13: Find the facilities usage by month, but not guests

In [188]:
with engine.connect() as con:
    rs = con.execute("SELECT f.name AS Facility, strftime('%m', b.starttime) AS Month, SUM(b.slots * 30) / 60 AS Hours_Booked \
                     FROM Bookings AS b \
                     LEFT JOIN Facilities AS f \
                     ON b.facid = f.facid \
                     WHERE memid != 0 \
                     GROUP BY Facility, strftime('%m', b.starttime)")
    
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

In [189]:
df

,Facility,Month,Hours_Booked
0,Badminton Court,07,82
1,Badminton Court,08,207
2,Badminton Court,09,253
3,Massage Room 1,07,83
4,Massage Room 1,08,158
5,Massage Room 1,09,201
6,Massage Room 2,07,4
7,Massage Room 2,08,9
8,Massage Room 2,09,14
9,Pool Table,07,55
